In [1]:
import pandas as pd
import numpy as np
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR 
#pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

 Generates association rules for the results found in the projects history last slice.

In [2]:
#df = pd.read_excel("historical_join.xlsx", keep_default_na=False)
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)

len(df)
df.head()

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df2 = df.copy()
df2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,7262dc3913045001cbc658a8474e9d295980c345,1,2010-07-14 06:36:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Activiti,Activiti,Enterprise Resource Planning,38b284602be9dcb6b4253cfa1c2f9420c002b56a,2,2010-08-03 08:31:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Activiti,Activiti,Enterprise Resource Planning,fbcdc18a66e7f7960a7ed966f87b1527c5c7551a,3,2010-09-02 09:45:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Activiti,Activiti,Enterprise Resource Planning,8d05dda18ce4c5db6f30cf01d7e0c2059b1a5388,4,2010-09-16 15:33:33+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Activiti,Activiti,Enterprise Resource Planning,a789ba2b30383777b4d0d81bbc62a90a24dbb032,5,2010-09-23 13:38:28+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31111,liferay,liferay-portal,Enterprise Resource Planning,fcd4cd6526c80613eca69927effe931bb6a91444,8209,2024-10-07 12:05:23+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0
31112,liferay,liferay-portal,Enterprise Resource Planning,cc0ab8103195e543056639c8c2883fcc187c33a2,8210,2024-10-07 17:58:58+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0
31113,liferay,liferay-portal,Enterprise Resource Planning,f75082029372ab28694650bd3fe9fae4d548351e,8211,2024-10-11 04:36:52+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0
31114,liferay,liferay-portal,Enterprise Resource Planning,a80a679a86d2867f985bafa510d9d12f1bf051a7,8212,2024-10-11 16:53:22+00:00,False,0,0,1,...,0,0,0,0,1,1,0,0,0,0


Grouping the lines to identify which projects that don't have DB clues

In [4]:
df2 = df.groupby(["owner", "name"]).sum()
df2

,,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
owner,name,,,,,,,,,,,,,,,,,,,,,
Activiti,Activiti,1540,0,0,0,32,0,0,0,13,0,...,0,0,0,0,0,0,0,0,0,0
Alluxio,alluxio,5886,0,0,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
AlmasB,FXGL,105,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AntennaPod,AntennaPod,703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xuxueli,xxl-job,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
yacy,yacy_search_server,5151,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zaproxy,zaproxy,666,0,0,0,0,0,0,0,36,0,...,0,0,0,0,0,0,0,0,0,0


Reseting the index column

In [5]:
df2 = df2.reset_index()
df2

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1540,0,0,0,32,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
2,AlmasB,FXGL,105,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AntennaPod,AntennaPod,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
358,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
360,zendesk,maxwell,105,0,0,0,0,0,0,0,...,0,0,0,0,0,7,0,0,0,0


New dataframe filtering only lines that last version projects

In [6]:
#df10 = df2.mask(df2 > 1, 1)
#df10 = df[(df['COMMITS'] == 10)]
#df10

# Função para selecionar o último commit
def select_last_commit(group):
    return group.iloc[-1]  # Seleciona o último commit

# Ordena os commits dentro de cada projeto (caso não estejam ordenados)
df_sorted = df.sort_values(by=['name', 'part_commit'])

# Aplica a função para pegar o último commit em cada grupo de projeto
df10 = df_sorted.groupby('name').apply(select_last_commit).reset_index(drop=True)

df10


,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,1667609eb4fb666fd9245f86ee1d7864644732f5,55,2024-02-14 12:26:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CellularPrivacy,Android-IMSI-Catcher-Detector,Security,80d6446ea02d66836305d8ffcd6405a805d023ae,13,2024-08-15 21:37:58+00:00,False,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Blankj,AndroidUtilCode,Software Development,d1ff4e3cd680473dc63b491a9795bd790a464949,11,2020-10-24 19:52:23+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AntennaPod,AntennaPod,Media,dd8bf381c4012558778abbefc14c1741dbab43a3,37,2024-05-18 17:26:39+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,Other,2fc649dc3398b1c762a85a803a43ce814519665b,37,2024-03-15 16:31:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,apache,zeppelin,Data Management,c5ccc8e584934fa4f96de486cd97312180147b1f,43,2024-09-29 15:41:29+00:00,False,0,0,0,...,0,1,0,0,0,1,0,0,0,0
357,openzipkin,zipkin,Monitoring,2ada407cd88f5360b4180e1df33a4dea0421774a,26,2024-02-11 12:02:55+00:00,False,0,0,0,...,0,1,0,0,0,0,0,0,0,0
358,apache,zookeeper,Infrastructure Management,6e746f8e881bfeb99cf0f74fa50a3d9c9d679229,25,2023-07-18 12:04:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zxing,zxing,Software Development,fc2f1c9b0f0218c72ec5cab0d4030b4abfb73945,36,2023-08-17 02:56:52+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues the principal dataframe 

In [7]:
#df2 = df2.iloc[ : , 1:]
#df3[(df3 == 0).all(axis=1)].index
df2 = df2.drop(df2[(df2.iloc[ : , 8:] == 0).all(axis=1)].index)
df2

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,1540,0,0,0,32,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alluxio,alluxio,5886,0,0,0,0,0,0,0,...,0,0,0,4,0,0,0,0,0,0
4,AnySoftKeyboard,AnySoftKeyboard,703,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Athou,commafeed,465,0,0,0,11,0,0,0,...,0,0,0,0,0,18,0,0,0,0
7,Atmosphere,atmosphere,1326,0,0,0,0,0,0,0,...,0,0,0,0,0,26,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,wildfly,wildfly,27495,0,0,0,143,0,0,0,...,0,0,0,0,0,0,0,0,0,0
357,xuxueli,xxl-job,120,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
358,yacy,yacy_search_server,5151,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
359,zaproxy,zaproxy,666,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Removing projects that don't have DB clues in the last version 

In [8]:
def select_last_commit(group):
    return group.iloc[-1]  # Seleciona o último commit

# Ordena os commits dentro de cada projeto (caso não estejam ordenados)
df_sorted = df.sort_values(by=['name', 'part_commit'])

# Aplica a função para pegar o último commit em cada grupo de projeto
df3 = df_sorted.groupby('name').apply(select_last_commit).reset_index(drop=True)

df3 = df3.drop(df3[(df3.iloc[:,8:] == 0).all(axis=1)].index) # deletando as linhas que possuem valor 0 em todas as colunas
df3

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,Activiti,Enterprise Resource Planning,1667609eb4fb666fd9245f86ee1d7864644732f5,55,2024-02-14 12:26:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,CellularPrivacy,Android-IMSI-Catcher-Detector,Security,80d6446ea02d66836305d8ffcd6405a805d023ae,13,2024-08-15 21:37:58+00:00,False,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,AxonFramework,AxonFramework,Infrastructure Management,bd3f94c81546c68b0138475ffd752d6ec710b79b,35,2024-02-16 12:45:30+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,opensourceBIM,BIMserver,Data Management,722fd6e1ae1dc0d2becdffc0207d5a5680d55c44,43,2020-10-13 07:37:36+00:00,False,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,BroadleafCommerce,BroadleafCommerce,Enterprise Resource Planning,d90401d8bdfa3c20b9a20b1570cd2b383a379a1f,66,2023-09-05 14:07:26+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,xuxueli,xxl-job,Automation,a4db25db6154b2d21913523f4f9db3259fa9ed2a,15,2022-05-21 05:15:21+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
354,yacy,yacy_search_server,Infrastructure Management,f1c70dce33f493414fb3e8e8911d78593b229725,101,2024-05-19 15:35:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
355,zaproxy,zaproxy,Security,4d95b8b54c234b2fb55e568de6519b47950fd7b4,36,2024-04-26 16:11:25+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
356,apache,zeppelin,Data Management,c5ccc8e584934fa4f96de486cd97312180147b1f,43,2024-09-29 15:41:29+00:00,False,0,0,0,...,0,1,0,0,0,1,0,0,0,0


Merging dataframe grouping and dataframe only last version with DB clues

In [9]:
#df3.merge(df4, on=['PROJECTS'], how='outer', suffixes=['', '_'], indicator=True)
df_mer= pd.merge(df2, df3, on= 'name', how='outer', suffixes=['', '_'], indicator=True)
df_mer

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Cassandra_,Couchbase_,Etcd_,PostGIS_,Redis_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
0,Activiti,Activiti,1540.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
1,Alluxio,alluxio,5886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,both
2,AnySoftKeyboard,AnySoftKeyboard,703.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,Athou,commafeed,465.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
4,Atmosphere,atmosphere,1326.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,yacy,yacy_search_server,5151.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
228,zaproxy,zaproxy,666.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,both
229,zendesk,maxwell,105.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,both
230,NaN,Android-IMSI-Catcher-Detector,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,right_only


Filtering diference in the last version

In [10]:
df5 = df_mer[(df_mer['_merge'].str.startswith('left'))]
df5

,owner,name,part_commit,isLast,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,...,Cassandra_,Couchbase_,Etcd_,PostGIS_,Redis_,Join_Ignite-Sql_,Join_Ignite-NoSql_,Join_Virtuoso-Sql_,Join_Virtuoso-NoSql_,_merge
2,AnySoftKeyboard,AnySoftKeyboard,703.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
15,JodaOrg,joda-time,231.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
16,M66B,NetGuard,630.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
26,SpigotMC,BungeeCord,171.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
29,Twitter4J,Twitter4J,171.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
38,apache,avro,861.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
58,apache,nutch,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
59,apache,poi,7875.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
61,apache,rocketmq,351.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
68,apache,struts,1225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


Merging dataframe only lines that last version projects and dataframe only lines in the last version and not DB clues

In [11]:
df_mer2= pd.merge(df10, df5.iloc[:,:2], on= 'name', how='outer', suffixes=['', '_'], indicator=True)
df_mer2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
0,Activiti,Activiti,Enterprise Resource Planning,1667609eb4fb666fd9245f86ee1d7864644732f5,55,2024-02-14 12:26:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
1,CellularPrivacy,Android-IMSI-Catcher-Detector,Security,80d6446ea02d66836305d8ffcd6405a805d023ae,13,2024-08-15 21:37:58+00:00,False,0,1,0,...,0,0,0,0,0,0,0,0,NaN,left_only
2,Blankj,AndroidUtilCode,Software Development,d1ff4e3cd680473dc63b491a9795bd790a464949,11,2020-10-24 19:52:23+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
3,AntennaPod,AntennaPod,Media,dd8bf381c4012558778abbefc14c1741dbab43a3,37,2024-05-18 17:26:39+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
4,AnySoftKeyboard,AnySoftKeyboard,Other,2fc649dc3398b1c762a85a803a43ce814519665b,37,2024-03-15 16:31:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,AnySoftKeyboard,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,apache,zeppelin,Data Management,c5ccc8e584934fa4f96de486cd97312180147b1f,43,2024-09-29 15:41:29+00:00,False,0,0,0,...,0,0,0,1,0,0,0,0,NaN,left_only
357,openzipkin,zipkin,Monitoring,2ada407cd88f5360b4180e1df33a4dea0421774a,26,2024-02-11 12:02:55+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only
358,apache,zookeeper,Infrastructure Management,6e746f8e881bfeb99cf0f74fa50a3d9c9d679229,25,2023-07-18 12:04:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
359,zxing,zxing,Software Development,fc2f1c9b0f0218c72ec5cab0d4030b4abfb73945,36,2023-08-17 02:56:52+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,left_only


Filtering only lines in the last version and not DB clues

In [12]:
df6 = df_mer2[(df_mer2['_merge'].str.startswith('bo'))]
df6

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
4,AnySoftKeyboard,AnySoftKeyboard,Other,2fc649dc3398b1c762a85a803a43ce814519665b,37,2024-03-15 16:31:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,AnySoftKeyboard,both
6,asciidocfx,AsciidocFX,Media,1c9442bafbb505446e7548307d3908e8ec123f7e,13,2023-07-30 09:08:18+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,asciidocfx,both
11,SpigotMC,BungeeCord,Game,c3f228f626af6da1dc126e7f96062cfc44d5b10f,18,2024-02-02 00:16:46+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,SpigotMC,both
27,M66B,NetGuard,Security,132e4c8d4e78194d682f2141a4579bde65c7cb2e,35,2023-11-01 06:34:49+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,M66B,both
33,opentripplanner,OpenTripPlanner,Data Management,178ebec96bbb2d611f5b2f62979bcb1a29d4b716,71,2024-10-03 18:50:32+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,opentripplanner,both
46,haraldk,TwelveMonkeys,Media,dd174c7f7ee167930afca01770b9bba84846a3aa,15,2024-08-06 08:29:24+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,haraldk,both
47,Twitter4J,Twitter4J,Media,9d705c33235b7358d2161bd03c7760fbc6919532,18,2022-10-11 13:53:37+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,Twitter4J,both
69,apache,avro,Data Management,2d3029ddae1ade2769bcb8b30e597db25dd56d1f,41,2024-09-23 11:18:54+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
77,bisq-network,bisq,Cryptocurrency,fcb625cbb430a06736c87948a56554dc3f080add,53,2024-03-27 09:13:19+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,bisq-network,both
79,bitcoinj,bitcoinj,Cryptocurrency,0080f79a051bf47dda1b62927dee2160cf6cba34,50,2024-08-29 20:51:42+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,bitcoinj,both


Concatening dataframes

In [13]:
df_concat2 =  pd.concat([df3, df6]).drop_duplicates()
df_concat2

,owner,name,domain,sha1,part_commit,date_commit,isLast,SapHana,Realm,SAP SQL Anywhere,...,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql,owner_,_merge
0,Activiti,Activiti,Enterprise Resource Planning,1667609eb4fb666fd9245f86ee1d7864644732f5,55,2024-02-14 12:26:10+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,CellularPrivacy,Android-IMSI-Catcher-Detector,Security,80d6446ea02d66836305d8ffcd6405a805d023ae,13,2024-08-15 21:37:58+00:00,False,0,1,0,...,0,0,0,0,0,0,0,0,NaN,NaN
7,AxonFramework,AxonFramework,Infrastructure Management,bd3f94c81546c68b0138475ffd752d6ec710b79b,35,2024-02-16 12:45:30+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
8,opensourceBIM,BIMserver,Data Management,722fd6e1ae1dc0d2becdffc0207d5a5680d55c44,43,2020-10-13 07:37:36+00:00,False,0,0,0,...,0,0,0,1,0,0,0,0,NaN,NaN
9,BroadleafCommerce,BroadleafCommerce,Enterprise Resource Planning,d90401d8bdfa3c20b9a20b1570cd2b383a379a1f,66,2023-09-05 14:07:26+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,spring-cloud,spring-cloud-netflix,Infrastructure Management,ad002b4da2de63e4512d70e9f888a47a3d6d7f07,21,2024-03-07 16:47:04+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,spring-cloud,both
328,apache,struts,Software Development,03126c01bfd07c88f0594275b1dd1b5d73ba8ea1,49,2024-05-13 04:50:20+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,apache,both
329,swagger-api,swagger-core,Software Development,95c8253c5239f6e980acd0b2998d037b35eb8761,15,2024-04-09 08:50:35+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,swagger-api,both
348,eclipse-vertx,vert.x,Infrastructure Management,b2f2eb3bb19770898a4b2e47d33939a8251be94d,36,2024-07-10 11:49:47+00:00,False,0,0,0,...,0,0,0,0,0,0,0,0,eclipse-vertx,both


In [14]:
#df3 = df3.iloc[ : , 2:] #desconiderandcoluna commits 
#df3[(df3 == 0).all(axis=1) # filtra as linhas que possuem valor igual a 0 em todas as colunas


In [15]:
#df2 = df2.astype(np.uint8)

In [16]:
df_concat2.drop(["owner","name", "domain", "sha1", "part_commit", "date_commit", "isLast", "owner_", "_merge"], axis=1, inplace=True)
df_concat2

,SapHana,Realm,SAP SQL Anywhere,FileMaker,ArangoDB,Informix,HyperSQL,CouchDB,Riak KV,Neo4j,...,Firebird,Cassandra,Couchbase,Etcd,PostGIS,Redis,Join_Ignite-Sql,Join_Ignite-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
348,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ASSOCIATION RULES

In [17]:
frq_items = apriori(df_concat2, min_support = 0.021, use_colnames = True)

In [18]:
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)

In [19]:
# Sorting the rules by the lift measure 
rules = rules.sort_values('lift', ascending =False)
print(rules.head())

                                              antecedents  \
778154  (MS SQL Server, PostgreSQL, HyperSQL, SAP Adap...   
756528  (MS SQL Server, MySQL, HyperSQL, SAP Adaptive ...   
833727     (SAP Adaptive Server, MS SQL Server, Firebird)   
833728        (SAP SQL Anywhere, MS SQL Server, Firebird)   
756507  (MS SQL Server, MySQL, Oracle, HyperSQL, SAP A...   

                                              consequents  antecedent support  \
778154  (SQLite, MySQL, Oracle, SAP SQL Anywhere, Info...            0.021645   
756528       (SQLite, Oracle, SAP SQL Anywhere, Informix)            0.021645   
833727  (SQLite, MySQL, Informix, Oracle, SAP SQL Anyw...            0.021645   
833728  (SQLite, MySQL, Informix, Oracle, SAP Adaptive...            0.021645   
756507           (SQLite, SAP SQL Anywhere, H2, Informix)            0.021645   

        consequent support   support  confidence  lift  leverage  conviction  
778154            0.025974  0.021645         1.0  38.5  0.021083   

In [20]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
778154,"(MS SQL Server, PostgreSQL, HyperSQL, SAP Adap...","(SQLite, MySQL, Oracle, SAP SQL Anywhere, Info...",0.021645,0.025974,0.021645,1.000000,38.500000,0.021083,inf
756528,"(MS SQL Server, MySQL, HyperSQL, SAP Adaptive ...","(SQLite, Oracle, SAP SQL Anywhere, Informix)",0.021645,0.025974,0.021645,1.000000,38.500000,0.021083,inf
833727,"(SAP Adaptive Server, MS SQL Server, Firebird)","(SQLite, MySQL, Informix, Oracle, SAP SQL Anyw...",0.021645,0.025974,0.021645,1.000000,38.500000,0.021083,inf
833728,"(SAP SQL Anywhere, MS SQL Server, Firebird)","(SQLite, MySQL, Informix, Oracle, SAP Adaptive...",0.021645,0.025974,0.021645,1.000000,38.500000,0.021083,inf
756507,"(MS SQL Server, MySQL, Oracle, HyperSQL, SAP A...","(SQLite, SAP SQL Anywhere, H2, Informix)",0.021645,0.025974,0.021645,1.000000,38.500000,0.021083,inf
...,...,...,...,...,...,...,...,...,...
24502,"(SQLite, H2, PostgreSQL)",(Redis),0.073593,0.350649,0.025974,0.352941,1.006536,0.000169,1.003542
87980,"(SQLite, H2, PostgreSQL, MySQL)",(Redis),0.073593,0.350649,0.025974,0.352941,1.006536,0.000169,1.003542
23336,(Redis),"(SQLite, H2, MySQL)",0.350649,0.073593,0.025974,0.074074,1.006536,0.000169,1.000519
88005,(Redis),"(SQLite, H2, PostgreSQL, MySQL)",0.350649,0.073593,0.025974,0.074074,1.006536,0.000169,1.000519


In [21]:
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules['consequente_len'] = rules['consequents'].apply(lambda x: len(x))

Filtering only the rules of size 2, that is, 1 antecedent and 1 consequent. 

In [22]:
rules_filter = rules[(rules['antecedent_len'] == 1) & (rules['consequente_len'] == 1)]

Showing all filtered rules.

In [23]:
rules_filter

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequente_len
404,(Ingres),(Firebird),0.021645,0.047619,0.021645,1.000000,21.000000,0.020614,inf,1,1
405,(Firebird),(Ingres),0.047619,0.021645,0.021645,0.454545,21.000000,0.020614,1.793651,1,1
67,(Informix),(MS Access),0.043290,0.030303,0.021645,0.500000,16.500000,0.020333,1.939394,1,1
66,(MS Access),(Informix),0.030303,0.043290,0.021645,0.714286,16.500000,0.020333,3.348485,1,1
50,(SAP Adaptive Server),(SAP SQL Anywhere),0.060606,0.064935,0.060606,1.000000,15.400000,0.056671,inf,1,1
...,...,...,...,...,...,...,...,...,...,...,...
226,(H2),(DynamoDB),0.389610,0.125541,0.056277,0.144444,1.150575,0.007365,1.022095,1,1
55,(Redis),(SAP SQL Anywhere),0.350649,0.064935,0.025974,0.074074,1.140741,0.003205,1.009870,1,1
54,(SAP SQL Anywhere),(Redis),0.064935,0.350649,0.025974,0.400000,1.140741,0.003205,1.082251,1,1
489,(Redis),(SAP Adaptive Server),0.350649,0.060606,0.021645,0.061728,1.018519,0.000394,1.001196,1,1


In [24]:
HISTORICAL_RULES_FILE = RESOURCE_DIR + os.sep + 'historical_rulesv10.xlsx'    #'seq_patterns/testes/historical_rules_commit10_197reg_021.xlsx'
rules_filter = rules_filter.to_excel(HISTORICAL_RULES_FILE, index=False)